In [1]:
from models import GCN
from ogb.nodeproppred import DglNodePropPredDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl

d:\Files\Coding\ml\EfficientGNN\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = 'cpu'
activation = nn.ReLU()
epochs = 100
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4
n_layers = 2

In [3]:
data = DglNodePropPredDataset('ogbn-arxiv', root='dataset/')
g, labels = data[0]
labels = labels[:, 0]
g.ndata['label'] = labels
g = dgl.add_reverse_edges(g)
features = g.ndata['feat']
idx_split = data.get_idx_split()
train_mask = idx_split['train']
val_mask = idx_split['valid']
test_mask = idx_split['test']
in_feats = features.shape[1]
n_classes = (labels.max() + 1).item()

In [4]:
model = GCN(in_feats, 16, n_classes, 2, activation)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [5]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

In [6]:
train(model, g, features, labels, train_mask, val_mask, test_mask)

Epoch 01, Loss: 3.6816, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 02, Loss: 3.5971, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 03, Loss: 3.4708, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 04, Loss: 3.3231, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 05, Loss: 3.1880, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 06, Loss: 3.0999, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 07, Loss: 3.0546, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 08, Loss: 3.0259, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 09, Loss: 2.9899, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 10, Loss: 2.9468, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 11, Loss: 2.9036, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 12, Loss: 2.8619, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 13, Loss: 2.8208, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 14, Loss: 2.7818, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 15, Loss: 2.7450, Train: 0.0000, Val: 0.0000, Test: 0.0000
Epoch 16, Loss: 2.7099, T